In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

# Big(ish) Data Handling

## Data Storage, Data Formats, Out-of-core processing and Data Accessing Strategies

# Table of Contents
* [Big(ish) Data Handling](#Big%28ish%29-Data-Handling)
	* [Data Storage, Data Formats, Out-of-core processing and Data Accessing Strategies](#Data-Storage,-Data-Formats,-Out-of-core-processing-and-Data-Accessing-Strategies)
		* [Learning Outcomes](#Learning-Outcomes)
			* [Massey Lab Instructions](#Massey-Lab-Instructions)
* [Splitting large files into smaller chunks - the naive approach](#Splitting-large-files-into-smaller-chunks---the-naive-approach)
* [Optimising Pandas DataFrame Accesses](#Optimising-Pandas-DataFrame-Accesses)
* [Strategies for Processing Large Datasets](#Strategies-for-Processing-Large-Datasets)
* [My dataset is bigger than my RAM](#My-dataset-is-bigger-than-my-RAM)
	* [HDF5](#HDF5)
		* &nbsp;
			* [In the above:](#In-the-above:)
		* [Read subsets of data from disk using HDF5](#Read-subsets-of-data-from-disk-using-HDF5)
* [Out-of-core processing with the Dask DataFrame](#Out-of-core-processing-with-the-Dask-DataFrame)
	* &nbsp;
		* [From now on we will pretend that our dataset is > RAM or large enough to take up too much of our RAM. Working with an actual dataset that is > RAM, will slow down all the examples in class considerably, but this is left as an exercise for you to perform out of class.](#From-now-on-we-will-pretend-that-our-dataset-is->-RAM-or-large-enough-to-take-up-too-much-of-our-RAM.-Working-with-an-actual-dataset-that-is->-RAM,-will-slow-down-all-the-examples-in-class-considerably,-but-this-is-left-as-an-exercise-for-you-to-perform-out-of-class.)
		* [Compare CSV to HDF5 speeds](#Compare-CSV-to-HDF5-speeds)
* [Sampling](#Sampling)
	* &nbsp;
		* [Density Map of NYC Rides](#Density-Map-of-NYC-Rides)
* [Store text/string data more efficiently with categoricals](#Store-text/string-data-more-efficiently-with-categoricals)
	* &nbsp;
		* [Let's compare again our categorised HDF5, with the previous execution times:](#Let's-compare-again-our-categorised-HDF5,-with-the-previous-execution-times:)
* [More Efficient On-Disk Storage Combined with Indexes](#More-Efficient-On-Disk-Storage-Combined-with-Indexes)
* [SQL Lite](#SQL-Lite)
	* [Summary of Strategies For Dealing with Inconvenient Datasets](#Summary-of-Strategies-For-Dealing-with-Inconvenient-Datasets)
* [Conclusion](#Conclusion)


### Learning Outcomes

At the end of this lecture, you should be able to:

* work with 'inconveniently large' datasets
* chunk large datasets 
* convert datasets to HDF5 
* perform out-of-core processing
* optimise dataset storage
* interface with sqlite datasets



This notebook will use the NYC Taxi Trip 2013-2014 dataset made up of 12 files, one for each month. The complete dataset is about 31GB. In the interest of time during class, the examples here will only use a portion of the data from the month of January. You are encouraged to employ the strategies below on the entire dataset in your own time. 

Dataset and selected examples drawn from http://www.andresmh.com/nyctaxitrips/. More info on the NYC Taxi Trip 2013-2014 dataset can be found here http://chriswhong.com/open-data/foil_nyc_taxi/ and http://hafen.github.io/taxi/#background. Selected material on Dask usage, sourced from https://github.com/dask/dask-tutorial 

#### Massey Lab Instructions

1. download trip_data_1a.csv and have it ready for class (especially if using a laptop)
2. save your file in C:/I folder because your H drive is likely to run out of space
3. write all files to the C drive for the same reason as above


------------------------

In [ ]:
!pip install ipython_memwatcher

In [ ]:
from ipython_memwatcher import MemWatcher
mw = MemWatcher()
mw.start_watching_memory()

# Limitations of Pandas 

According to Wes McKinney, the author of Python Pandas, there are some serious limitations inherent in Pandas when it comes to dealing with larger datasets that were not considered when the project to develop Pandas was initiated:

> To put it simply, we weren't thinking about analyzing 100 GB or 1 TB datasets in 2011. Nowadays, my rule of thumb for pandas is that you should have 5 to 10 times as much RAM as the size of your dataset. So if you have a 10 GB dataset, you should really have about 64, preferably 128 GB of RAM if you want to avoid memory management problems. This comes as a shock to users who expect to be able to analyze datasets that are within a factor of 2 or 3 the size of their computer's RAM.

>> **pandas rule of thumb: have 5 to 10 times as much RAM as the size of your dataset**

> There are additional, hidden memory killers in the project, like the way that we use Python objects (like strings) for many internal details, so **it's not unusual to see a dataset that is 5GB on disk take up 20GB or more in memory**. It's an overall bad situation for large datasets.

Source: http://wesmckinney.com/blog/apache-arrow-pandas-internals/

# Splitting large files into smaller chunks - the naive approach



In [ ]:
file_path = r'D:/158739_2019/2019_Python_notebooks/Block_2/datasets/'

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt


Let's assume that the file we are dealing with in this example cannot fit into out working memory (RAM). What can we do?....

Well, given a very large csv file, often the most simplistic approach to working with it is to split it into multiple smaller csv file chunks, and then process each one in turn.

Here we will re-write each chunk to disk.

In [ ]:
import zipfile
z = zipfile.ZipFile(file_path + 'trip_data_1a.zip')


In [ ]:

# the original trip_data_1.csv data file has over 14M records - 
# the trip_data_1a.csv has 5M rows which we will divide here into three chunks
chunksize = 2000000
i = 0
f = ['a','b','c']
for df in pd.read_csv(z.open('trip_data_1a.csv'), chunksize=chunksize):
        df.to_csv(file_path + 'trip_data_1a_' + f[i] + '.csv', index=None)
        i += 1

OR

In [ ]:
# the original trip_data_1.csv data file has over 14M records - 
# the trip_data_1a.csv has 5M rows which we will divide here into three chunks
chunksize = 2000000
i = 0
f = ['a','b','c']
for df in pd.read_csv(file_path + 'trip_data_1a.csv', chunksize=chunksize):
        df.to_csv(file_path + 'trip_data_1a_' + f[i] + '.csv', index=None)
        i += 1

# Optimising Pandas DataFrame Accesses 

When performing heavily intensive computations on dataframes of a large size, it is imperative that some form of profiling and optimisation is carried out.

The way we process and access pandas dataframes can have significant effects on the runtime performance that differs in orders of magnitude between various options.  

Since we can fit all our data into memory, let's do this in order to explore some pandas performance characteristics.

http://jose-coto.com/slicing-methods-pandas
https://stackoverflow.com/questions/28757389/loc-vs-iloc-vs-ix-vs-at-vs-iat

In [ ]:
#read it as a zipped file or a csv
df = pd.read_csv(z.open('trip_data_1a.csv'), infer_datetime_format=True, parse_dates=['pickup_datetime', 'dropoff_datetime'])

OR

In [ ]:
df = pd.read_csv(file_path + 'trip_data_1a.csv', infer_datetime_format=True, parse_dates=['pickup_datetime', 'dropoff_datetime'])

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.info()

In [ ]:
#df = pd.read_hdf('../datasets/trip_data_1a.h5', '/data', start=0, stop=1000000)
#df.tail()

We will consider looping through a dataframe that has 1M rows.

iat - position based works similarly to iloc.

Advantage over iloc is that this is faster.

Disadvantage is that you can't use arrays for indexers. Cannot assign new indices and columns.

In [ ]:
def dataframe_access_using_iat(df):
    for i in range(len(df)):
        df.iat[i, 0]
        
%time dataframe_access_using_iat(df[:1000000])

In [ ]:
len(df)

In [ ]:
# example
df.iat[10000000, 0] = 0

In [ ]:
# example
df.iat[ [0,1] , 0]

In [ ]:
# example
df.iat[0, 'medallion']

at - label based works very similarly to loc for scalar indexers.

Advantage over loc is that this is faster. 

Disadvantage is that you can't use arrays for indexers. Cannot assign new indices and columns.

In [ ]:
def dataframe_access_using_at(df):
    for i in range(len(df)):
        df.at[i, 'medallion']

%time dataframe_access_using_at(df[:1000000])

In [ ]:
# example
df.at[0, 0]

In [ ]:
# example
df.at[10000000, 'medallion'] = 0

iloc - position based
Similar to loc except with positions rather that index values. However, you cannot assign new columns or indices.

In [ ]:
def dataframe_access_using_iloc(df):
    for i in range(len(df)):
        df.iloc[i, 0]

%time dataframe_access_using_iloc(df[:1000000])

In [ ]:
# example
df.iloc[ [0,1] , 0] 

In [ ]:
# example
df.iloc[ [0,1], 'medallion']

 ix - It accepts label based or index positional arguments.

**Exercise:** *.ix* operator is now deprecated but the *.loc* access operator still remains as untested. Using the approach above, create a test for this operator below.

loc - label based
Allows you to pass 1-D arrays as indexers. Arrays can be either slices (subsets) of the index or column, or they can be boolean arrays which are equal in length to the index or columns.

Special Note: when a scalar indexer is passed, loc can assign a new index or column value that didn't exist before.

# Strategies for Processing Large Datasets 

Most python modules and machine learning algorithms assume that all the data can be loaded into memory.

This is increasingly becoming unrealistic with the amount of data that is being generated, stored and used for data analysis.

This creates considerable challenges. When the data is in the order of terabytes, then Hadoop distributed processing on clusters of commodity machines is the standard approach.

Such infrastructure is not always available. 

When the datasets are in tens of gigabytes in size, Hadoop is an overkill and is often not the preferred option. 

Data that is larger than the size of a machine's RAM capacity is not Big Data, but it is large and inconvenient to work with. The challenges with this kind of data are:

1. execution runtime
2. working with libraries that enable processing the data without bringing all of it into RAM (out-of-core processing)
3. ensuring that dataset file formats are optimised for out-of-core processing

First of all, since all of our data fits into the working memory, we can perform pretty fast computations on millions of records since the CPU is not having to access the disk.

In [ ]:
%time df.passenger_count.mean()

In [ ]:
%time df.trip_time_in_secs.mean()

In [ ]:
%time df.describe()

# My dataset is bigger than my RAM

If this happens and you only have access to the tools covered so far, then you can resort to chunking/splitting the dataset as shown above and working on each part in turn.

This is extremely tedious. Do not do this if you can avoid it. Usually you can.

There are far better strategies available to us. 

**Out-of-core processing** allows us to get around this problem is. However, it can still be very slow when the dataset is stored on disk in a non-binary format like csv. 

Files that are not saved in a **binary compressed format**, are not stored optimally and must also be interpreted as they are read from disk, which slows down the read time. 

Therefore, saving large files in a binary format instead of a text format like a csv, will ensure that the data is stored and read in future in a much more efficient manner.

Saving datasets in specific binary formats also allows us to **filter and read into memory only subsets of data** that meet a required condition, which can significantly limit the RAM requirement. 

## HDF5

**HDF5** (http://www.h5py.org/) is a binary format for efficiently storing and accessing data which makes it much more suitable for sharing and out-of-core processing.

Therefore, when working with very large files, convert and store them in a binary format like HDF5.

HDF5 stands for Hierarchical Data Format. It is a multipurpose hierarchical container format capable of storing large **numerical datasets** with their metadata. The specification is open and the tools are open source. 

It supports an unlimited variety of datatypes, and is designed for flexible and efficient I/O and for high volume and complex data. HDF5 is portable and is extensible.

An HDF5 file contains a hierarchy of numerical arrays (or datasets) organized within groups.

A dataset can be stored in two ways: contiguously or chunked. The former stores a dataset in a contiguous buffer in the file, while the latter splits it uniformly in rectangular chunks organized in a B-tree.

HDF5 also supports lossless compression of datasets.

In [ ]:
df = pd.read_csv(file_path + 'trip_data_1a.csv', infer_datetime_format=True, parse_dates=['pickup_datetime', 'dropoff_datetime'])

In [ ]:
%time df.to_hdf(file_path + 'trip_data_1a.h5', '/data', format='table', mode='w', data_columns=['trip_distance'])

#### In the above: 

*format='table'* specifies that the file is searchable

*mode='w' specifies write/create/overwrite 
 
*data_columns=['trip_distance']* cretes an index on the specified column so that we can select/search data on

*/data is the object under which a given dataset is stored and is defined as a file path, you can define this name to whatever is suitable for the domain

In [ ]:
hdf = pd.HDFStore(file_path + 'trip_data_1a.h5')
hdf

In [ ]:
hdf['/data'].shape

Datasets can be appended (using append) to the existing dataset in the HDF5 format, or new objects can be created within the same file under a different group path:

In [ ]:
hdf.put('tables/t1', pd.DataFrame(np.random.rand(20,5)))
hdf.put('tables/t2', pd.DataFrame(np.random.rand(10,3)))
hdf.put('data/t1', pd.DataFrame(np.random.rand(15,2)))
hdf

In [ ]:
hdf

In [ ]:
hdf.close()

In [ ]:
#release memory
#del df

### Read subsets of data from disk using HDF5

If we have a really large file that cannot fit into RAM, but has been converted into an HDF5 format, we can read into memory only portions of it that are of interest to us an to our immediate processing needs. 

We can bring into RAM only selected columns, and/or rows that satisfy a certain criterion. We can do this is if the HDF5 file is searchable, and we can specify filter constraints during file read I/O on the indexed columns.

In [ ]:
df = pd.read_hdf(file_path + 'trip_data_1a.h5', '/data', columns=['medallion','pickup_datetime', 'trip_distance'], where='trip_distance>10.0')
df.head()

In [ ]:
len(df)

Notice that we imported a subset of columns above, based on a condition specified by the *where* argument.

**Exercise:** Write a query that reads in data from the above file where the trip distance is greater than 5 miles and smaller than 15 miles.

**Exercise:** Write a query that reads in data from the above file where the number of passengers is greater than 5.

You'll notice an error.

This is because the *passenger_count* filed is not indexed and is not searchable. 

**Exercise:** Re-write the trip_data_1a file into HDF5, enabling the search for *passenger_count*, then perform the above query again.

**Exercise:** Read data from the above file, where the index number is between 10000 and 20000.

# Out-of-core processing with the Dask DataFrame

### From now on we will pretend that our dataset is > RAM or large enough to take up too much of our RAM. Working with an actual dataset that is > RAM, will slow down all the examples in class considerably, but this is left as an exercise for you to perform out of class.

[`Dask`](https://dask.readthedocs.org/en/latest/) is a parallel computing library, designed to enable out-of-core processing. This means that most of the dataset can continue sitting on the hard disk while you are processing it, and it does not need to be read in it's entirety.

Dask has a DataFrame class which functions just like the pandas counterpart; however, the **Dask DataFrame does not have all the functionality of a pandas DataFrame**.

Dask can read large datasets from multiple file types.

We will read the same data from a csv and HDF5 files and compare the out-of-core performance efficiencies of the two file types.

We load this data into a dask dataframe using the `read_csv` function.  This has the exact same signature as `pandas.read_csv`.

In [ ]:
import dask.dataframe as dd

In [ ]:
%time ddf_csv = dd.read_csv(file_path + 'trip_data_1a.csv', \
                 parse_dates=['pickup_datetime', 'dropoff_datetime'])
ddf_csv.head()

In [ ]:
ddf_csv.dtypes

In [ ]:
%time ddf_hdf = dd.read_hdf(file_path + 'trip_data_1a.h5', '/data')
ddf_hdf.head()

In [ ]:
ddf_hdf.dtypes

In [ ]:
import sys
sys.getsizeof(df)

In [ ]:
import sys
sys.getsizeof(ddf_csv)

In [ ]:
import sys
sys.getsizeof(ddf_hdf)

Calling *.compute()* on a Dask DataFrame, converts the returned data into a pandas DataFrame.

### Compare CSV to HDF5 speeds

We do a simple computation that requires reading a column of our dataset and compare performance between CSV files and our newly created HDF5 file.

In [ ]:
%time ddf_csv.passenger_count.sum().compute()

In [ ]:
%time ddf_hdf.passenger_count.sum().compute()

In [ ]:
%time ddf_csv[['vendor_id','passenger_count']].groupby('vendor_id').count().compute()

In [ ]:
%time ddf_hdf[['vendor_id','passenger_count']].groupby('vendor_id').count().compute()

In [ ]:
ddf_hdf.head()

**Exercise:** Construct a groupby query that groups the data by driver (medallion) and sums up all the trips travelled, listing the top 10. Do this using csv and hdf files and compare the execution times.

In [ ]:
%time ddf_hdf[['medallion','trip_distance']].groupby(ddf_hdf.medallion).count().compute().sort_values(by='trip_distance', ascending=False).head(10)

# Sampling

Sometimes when you have an enormous amount of data, you do not need to visualise or compute all of it. As long as you have out-of-core processing capabilities, it is often enough just to sample the data instead of working with all of it.

**Let's do some more processing of our data, but this time using the sampling strategy to handle the data size challenge,  and let's observe performance differences between accessing CSV and HDF5 files.**

### Density Map of NYC Rides

We'll use Bokeh to plot out a density map of all start and end locations for all taxi rides.

Get sample of pickup and dropoff locations

First we'll pick out a few columns and use the `sample` method to pick out a random sample of 1% of all rides.  We'll then call `compute` on this data to convert the results into in-memory Pandas DataFrames that we can hand off to Bokeh for visualisation.

We'll track our progress with the `ProgressBar` diagnostic.

In [ ]:
from dask.diagnostics import ProgressBar

In [ ]:
sample = ddf_csv.sample(frac=0.01)
print(len(sample))
pickup = sample[['pickup_latitude', 'pickup_longitude']]
with ProgressBar():
    result = pickup.compute()  # compute to smaller Pandas DataFrame

In [ ]:
len(result)

In [ ]:
from bokeh.plotting import figure, show, output_notebook
output_notebook()

In [ ]:
p = figure(title="Pickup Locations")
p.scatter(result.pickup_longitude, result.pickup_latitude, size=3, alpha=0.2)
show(p)

In [ ]:
sample = ddf_hdf.sample(frac=0.01)
pickup = sample[['pickup_latitude', 'pickup_longitude']]
with ProgressBar():
    result = pickup.compute()  # compute to smaller Pandas DataFrame

In [ ]:
p = figure(title="Pickup Locations")
p.scatter(result.pickup_longitude, result.pickup_latitude, size=3, alpha=0.2)
show(p)

#  Store text/string data more efficiently with categoricals

As it turns out, a considerable portion of access time is consumed by accessing string data. This is because Pandas represents text with the **object dtype** which holds a normal Python string. 

Object dtypes are usually the main cause of slow code because **object dtypes run at Python speeds, not at Pandas’ normal C speeds**.

Pandas **categoricals** are a new and powerful feature that **encodes categorical data into a numerical representation** so that we can leverage Pandas’ fast C code on this kind of text data.

Given this, there are some strategies we can use in order to encode string data more efficiently which will **improve performance**.

The strategy will demonstrate how to convert string data into categoricals. *This works very well when the data does not have a large number of distinct string values for a given column*.

This takes a bit more time up front, but results in better performance.

More on categoricals at the [pandas docs](http://pandas-docs.github.io/pandas-docs-travis/categorical.html).

In [ ]:
ddf_hdf.head()

We will categorise the 'vendor_id' and 'store_and_fwd_flag' columns.



In [ ]:
# Categorize data
ddf_hdf = ddf_hdf.categorize(columns=['vendor_id','store_and_fwd_flag'])

In [ ]:
ddf_hdf.to_hdf(file_path + 'trip_data_1a_categorical.h5', '/data', format='table', mode='w', data_columns=['trip_distance'])

You should now notice that about a 25% reduction in disk space has been realized.

### Let's compare again our categorised HDF5, with the previous execution times:

(This may run slower on your computer depending on how full your RAM is from the previous datasets - try restarting the kernel and comparing the execution times from this point on without performing all the previous steps)

In [ ]:
import pandas as pd
import datetime as dt
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
from bokeh.plotting import figure, show, output_notebook
#import castra as c
output_notebook()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import time
import seaborn as sns
from pylab import rcParams

%matplotlib inline
rcParams['figure.figsize'] = 20, 10
rcParams['font.size'] = 20

rcParams['figure.dpi'] = 350
rcParams['lines.linewidth'] = 2
rcParams['axes.facecolor'] = 'white'
rcParams['patch.edgecolor'] = 'white'
rcParams['font.family'] = 'StixGeneral'

In [ ]:
file_path = r'D:/158739_2019/2019_Python_notebooks/Block_2/datasets/'

In [ ]:
ddf_csv = dd.read_csv(file_path + 'trip_data_1a.csv', parse_dates=['pickup_datetime', 'dropoff_datetime'])
ddf_hdf = dd.read_hdf(file_path + 'trip_data_1a_categorical.h5', '/data')

In [ ]:
ddf_hdf.head()

Compare with previous execution runtimes:

In [ ]:
%time ddf_hdf.passenger_count.sum().compute()

In [ ]:
%time len(ddf_hdf)

In [ ]:
%time ddf_hdf[['vendor_id','passenger_count']].groupby('vendor_id').count().compute()

In [ ]:
sample = ddf_hdf.sample(frac=0.01)
pickup = sample[['pickup_latitude', 'pickup_longitude']]

with ProgressBar():
    result = pickup.compute()  # compute to smaller Pandas DataFrame

To fully appreciate and highlight the performance improvement of using categoricals, we will perform queries on categorical data that is in RAM.

Let's consider the medallion column which has not been categorised yet. Let's load it into RAM and perform a query on the original:


In [ ]:
df = ddf_hdf[['medallion','trip_distance']].compute()

**Exercise:** Perform a groupby operation on this new dataframe and time the operation that groups the dataframe by medallion and sums the trip_distances for each medallion, and sorts the results in a descending order, showing the top ten medallions (taxis) by total distance travelled. 

**Exercise:** Convert the medallion columns into a category data type: 

**Exercise:** Perform the same group by operation as above. Time the execution and compare.

In [ ]:
del ddf_csv
del ddf_hdf

# More Efficient On-Disk Storage Combined with Columnar and Compression


We saw the significant speed improvement realised by resorting to HDF5. The disadvantage with HDF5 is that all the column data is stored together in contiguous blocks, which means that it is inefficient when a dataset comprises of numerous columns and frequently, only subsets of columns must be accessed.  

Therefore, there are more performance improvements that we can realise.

-----------------------------

Good on-disk storage for DataFrames generally has the following characteristics

1.  Binary storage format:  We don't store values as text that needs to be parsed as integers.  This rules out CSV and JSON.
2.  Categorical support:  Text is hard in Python.  Pandas Categoricals can speed this up when the text is often repeated.
3.  **Compression:  Light compression can often increase data bandwidth from disk.  Difficult to do right and depends largely on your data characteristics.**
2.  **Column Stores:  We store individual columns separately and in chunks so that queries on just a few columns can load a small fraction of the dataset.  This rules out typical use of HDF5.**

###   bcolz Project 

> bcolz provides columnar, chunked data containers that can be compressed either in-memory and on-disk. Column storage allows for efficiently querying tables, as well as for cheap column addition and removal. It is based on NumPy, and uses it as the standard data container to communicate with bcolz objects, but it also comes with support for import/export facilities to/from HDF5/PyTables tables and pandas dataframes.

> bcolz objects are compressed by default not only for reducing memory/disk storage, but also to improve I/O speed. The compression process is carried out internally by Blosc, a high-performance, multithreaded meta-compressor that is optimized for binary data (although it works with text data just fine too).

> bcolz can also use numexpr internally (it does that by default if it detects numexpr installed) or dask so as to accelerate many vector and query operations (although it can use pure NumPy for doing so too). numexpr/dask can optimize the memory usage and use multithreading for doing the computations, so it is blazing fast. This, in combination with carray/ctable disk-based, compressed containers, can be used for performing out-of-core computations efficiently, but most importantly transparently.

> By using compression, you can deal with more data using the same amount of memory, which is very good on itself. But in case you are wondering about the price to pay in terms of performance, you should know that nowadays memory access is the most common bottleneck in many computational scenarios, and that CPUs spend most of its time waiting for data. Hence, having data compressed in memory can reduce the stress of the memory subsystem as well.

> Furthermore, columnar means that the tabular datasets are stored column-wise order, and this turns out to offer better opportunities to improve compression ratio. This is because data tends to expose more similarity in elements that sit in the same column rather than those in the same row, so compressors generally do a much better job when data is aligned in such column-wise order. In addition, when you have to deal with tables with a large number of columns and your operations only involve some of them, a columnar-wise storage tends to be much more effective because minimizes the amount of data that travels to CPU caches.

> So, the ultimate goal for bcolz is not only reducing the memory needs of large arrays/tables, but also making bcolz operations to go faster than using a traditional data container like those in NumPy or Pandas. That is actually already the case in some real-life scenarios (see the notebook above) but that will become pretty more noticeable in combination with forthcoming, faster CPUs integrating more cores and wider vector units.

> source: https://github.com/Blosc/bcolz

Blosc is a compression library that is often used with bcolz:

> Blosc (http://blosc.org) is a high performance compressor optimized for binary data. It has been designed to transmit data to the processor cache faster than the traditional, non-compressed, direct memory fetch approach via a memcpy() OS call.

> Blosc works well for compressing numerical arrays that contains data with relatively low entropy, like sparse data, time series, grids with regular-spaced values, etc.

> python-blosc a Python package that wraps Blosc. python-blosc supports Python 2.7 and 3.4 or higher versions.

> source: https://github.com/Blosc/python-blosc

# SQL Lite

Sometimes it makes sense to store large data in SQLite (or other fully fledged Database Management Systems like MySQL, Postgres, MongoDB etc. who are running as a separate process ).

In [ ]:
import sqlite3
import datetime as dt

Converting our dataset to SQLite can be done by chunking.


In [ ]:
ddf_hdf.describe()

In [ ]:
df = pd.read_csv(file_path + 'trip_data_1a.csv', infer_datetime_format=True, parse_dates=['pickup_datetime', 'dropoff_datetime'])

In [ ]:
df.head()

In [ ]:
df = df.set_index('pickup_datetime')
df = df.sort_index()

In [ ]:
df.head()

In [ ]:
%time df.to_hdf(file_path + 'trip_data_1a_pickup_datetime_index.h5', '/data', format='table', mode='w', data_columns=['trip_distance'])

In [ ]:
ddf_datetime_index_hdf = dd.read_hdf(file_path + 'trip_data_1a_pickup_datetime_index.h5', '/data')
ddf_datetime_index_hdf.head()

In [ ]:
ddf_datetime_index_hdf.loc['2013-1-1'].compute()

In [ ]:
chunksize = 10
day = 1
date = '2013-1-'
time = ' 23:59:59'
total = 0

start = dt.datetime.now()
conn = sqlite3.connect(file_path + "trip_data_1a_a.sl3")
while (day <= 20):
    temp = ddf_datetime_index_hdf.loc[date + str(day) : date + str(day) + time].compute()
    temp.reset_index(inplace=True)
    temp.to_sql('TAXITRIPDATA', conn, if_exists='append', index=False)
    day += chunksize
    total += len(temp)
    print('{} seconds: completed {} rows inserted'.format((dt.datetime.now() - start).seconds, total))

In [ ]:
len(df['2013-1-1'])+len(df['2013-1-11'])

In [ ]:
df.head()

In [ ]:
conn.close()

Or, alternatively if you have a csv that needs to be converted into a sql file, this can also be done by chunking:

In [ ]:
chunksize = 100000
total = 0
start = dt.datetime.now()

conn2 = sqlite3.connect(file_path + "trip_data_1a_2.sl3")
for df in pd.read_csv(file_path + "trip_data_1a.csv", chunksize=chunksize):
    df.to_sql('TAXITRIPDATA', conn2, if_exists='append', index=False)
    total += len(df)
    print('{} seconds: completed {} rows inserted'.format((dt.datetime.now() - start).seconds, total))


In [ ]:
conn2.close()

In [ ]:
del df

There are modules like **odo** that also automate the conversion process from multiple data types. See [`Odo documentation`](http://odo.pydata.org/en/latest/uri.html)

Once a sqlite database file has been created, it can be opened and queried using the SQL language and out-of-core-processing within pandas as follows:

In [ ]:
conn = sqlite3.connect(file_path + "trip_data_1a_2.sl3")
curs = conn.cursor()

In [ ]:
df_sql = pd.read_sql_query('SELECT * FROM TAXITRIPDATA LIMIT 10', conn)
df_sql

In [ ]:
start = dt.datetime.now()
df_sql = pd.read_sql_query('SELECT medallion, pickup_datetime, dropoff_datetime, passenger_count '
                           'FROM TAXITRIPDATA '
                           'WHERE passenger_count > 5 '
                           'LIMIT 30', conn)
print('Operation took {} seconds.'.format((dt.datetime.now() - start).seconds))
df_sql.head(100)

Let's time the operation:

In [ ]:
start = dt.datetime.now()
df_sql = pd.read_sql_query('SELECT medallion, COUNT(*) as `num_journeys` '
                           'FROM TAXITRIPDATA '
                           'GROUP BY medallion '
                           'ORDER BY -num_journeys '
                           'LIMIT 30', conn)
print('Operation took {} seconds.'.format((dt.datetime.now() - start).seconds))
df_sql.head(100)

The performance is far from ideal.

It's possible to improve the access time by creating an index on a key column.

**WARNING:** This might take a bit of time... 

**Exercise:** Write the SQL statement that will create an index on the 'medallion' field':

**Exercise:** Re-execute the SQL query from the above example and compare the execution time with the newly set up index:

**Exercise:** Create a new index on another variable. Experiment with the above query to see if the new index has improved the performance.

Ensure that the database connection is closed once finished. 

In [ ]:
conn.close()

## Summary of Strategies For Dealing with Inconvenient Datasets

When dataset > RAM, we have a number of options in order to move forward and realise computational efficiency:

1. Chunking the dataset.
2. Using out-of-core processing libraries
3. Convert datasets to more efficient encodings for disk I/O operations during out-of-core processing
4. Filter binary datasets and bring into RAM only subsets of data
5. Look to optimise file encodings with the particular options they each provide - ie string categorisation 
6. Combine the above strategies with sampling. Use out-of-core processing to downsample before pass off the smaller dataset to `pandas` to process quickly in-memory.
7. Set an index on important columns
8. Use SQLite, or interface with a full-blown database management instance that is running in its own separate process

# Conclusion

Storage choices strongly impact performance.  

This becomes even more critical when the data is large.

Strategies for handling data > RAM were covered.

Key points:
- store large datasets in a binary format
- ensure that the binary format supports search/filtering on key columns
- categorise string columns for more efficient queries
- consider using Dask for out-of-core processing as well as other emerging libraries (Bcolz)
- consider converting your data into a SQLite format, or if needed, loading it up into a database management system (MySQL, Oracle, SQL Server, MongoDB, Postgres etc.)
- investigate how you can optimise your code


In [ ]:
%%javascript
require(['base/js/utils'],
function(utils) {
   utils.load_extensions('calico-spell-check', 'calico-document-tools', 'calico-cell-tools');
});